In [ ]:
def ShowImageFromByteArray(byte_array, width=300, height=300):
    from IPython.display import display, Image
    from PIL import Image as PILImage
    from io import BytesIO
    img = PILImage.open(BytesIO(byte_array))
    img = img.resize((width, height))
    b = BytesIO()
    img.save(b, format='png')
    display(Image(data=b.getvalue()))

In [2]:
def GetFilesXlsxInFolder(folder):
    import os
    files = os.listdir(folder)
    return [os.path.join(folder, f) for f in files if f.endswith(".xlsx")]
GetFilesXlsxInFolder("./")

['./surveillance-systems-overview-2017_4.xlsx',
 './surveillance-systems-overview-2018.xlsx',
 './Table-surveillance_systems_overview_2.xlsx',
 './Table-surveillance_systems_overview_2019_20210215.xlsx',
 './Table-surveillance_systems_overview_3 (1).xlsx',
 './Table-surveillance_systems_overview_3.xlsx',
 './Table-surveillance_systems_overview_for_2016_0.xlsx']

In [8]:
import pandas as pd

def ConvertXlsxToCsv(xlsx_file, csv_file):
    df = pd.read_excel(xlsx_file)
    df.to_csv(csv_file, index=False)

df = pd.read_excel(GetFilesXlsxInFolder("./")[0])
df.head()

,,Annual epidemiological report for 2017
0,NaN,Surveillance systems overview for 2017
1,NaN,NaN
2,NaN,This spreadsheet contains all surveillance sys...
3,NaN,The included diseases are listed at the bottom...
4,NaN,NaN


['./surveillance-systems-overview-2017_4.xlsx',
 './surveillance-systems-overview-2018.xlsx',
 './Table-surveillance_systems_overview_2.xlsx',
 './Table-surveillance_systems_overview_2019_20210215.xlsx',
 './Table-surveillance_systems_overview_3 (1).xlsx',
 './Table-surveillance_systems_overview_3.xlsx',
 './Table-surveillance_systems_overview_for_2016_0.xlsx']

In [2]:
import pandas as pd
df = pd.read_csv("ECDC_surveillance_data_Chikungunya_virus_disease.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'ECDC_surveillance_data_Chikungunya_virus_disease.csv'

In [4]:
import pycountry

def getCountryCode(x):
    try:
        return pycountry.countries.get(name=x).alpha_3
    except:
        return None

#df["CountryCode"] = df["RegionName"].apply(getCountryCode)

#df["CountryCode"]

In [5]:
print(getCountryCode("United States"))

US


In [37]:
s_df = df[df["Location"].notna()]

In [40]:
import pandas as pd

df = pd.read_csv("data/TotalCaseEurope.csv")

df = df[['Time', 'RegionName', 'NumValue']]

df = df[df["RegionName"] != "EU/EEA"]
df = df[df["RegionName"] != "EU"]
df["CountryCode"] = df["RegionName"].apply(getCountryCode)

df.to_csv("cleaned/TotalCasesEurope.csv", index=False)

In [58]:
import pandas as pd

def formalizeAmericanDf(year):
    df = pd.read_csv("data/TotalCaseAmerica"+str(year)+".csv")

    #Keep only interesting rows
    df = df[['Country or Subregion', 'Measure Values']]
    #Rename for consistency
    df = df.rename(columns={"Country or Subregion": "RegionName", "Measure Values": "NumValue"}, errors="raise")
    #Add row for the year
    df["Time"]=[str(year)]*len(df)
    #Correct the name of the United States
    df.loc[df["RegionName"] == "United States of America", 'RegionName'] = "United States"
    #Add the country code
    df["CountryCode"] = df["RegionName"].apply(getCountryCode)
    #Remove Weird countries or agglomerations
    df = df[df["CountryCode"].notna()]

    return df

tmp = []
for year in range(2019, 2023):
    tmp.append(formalizeAmericanDf(year))
pd.concat(tmp).to_csv("cleaned/TotalCasesAmerica.csv", index=False)

In [59]:
from geopy.geocoders import Nominatim
import numpy as np

geolocator = Nominatim(user_agent="chickungunyalocation")
def geolocate(country):
    try:
        loc = geolocator.geocode(country)
        return (loc.latitude, loc.longitude)
    except:
        return np.nan
europe = pd.read_csv("cleaned/TotalCasesEurope.csv")
america = pd.read_csv("cleaned/TotalCasesEurope.csv")
europe["Location"] = europe["CountryCode"].apply(geolocate)
america["Location"] = america["CountryCode"].apply(geolocate)

In [66]:

europe = pd.read_csv("cleaned/TotalCasesEurope.csv")
europe.iloc[0]['Location'].split(", ")[0][1:]
europe.iloc[0]['Location'].split(", ")[1][:-1]


'14.12456'

In [ ]:
def groupbysumdf(df):

In [100]:
europe = pd.read_csv("cleaned/TotalCasesEurope.csv")
europe[['CountryCode', "RegionName", "Location", "NumValue"]].groupby(['CountryCode', "RegionName", "Location"]).sum().reset_index()

,CountryCode,RegionName,Location,NumValue
0,AT,Austria,"(47.59397, 14.12456)",44.0
1,BE,Belgium,"(50.6402809, 4.6667145)",265.0
2,CZ,Czechia,"(49.7439047, 15.3381061)",32.0
3,DE,Germany,"(51.1638175, 10.4478313)",668.0
4,EE,Estonia,"(58.7523778, 25.3319078)",0.0
5,ES,Spain,"(39.3260685, -4.8379791)",764.0
6,FI,Finland,"(63.2467777, 25.9209164)",38.0
7,FR,France,"(46.603354, 1.8883335)",1167.0
8,GB,United Kingdom,"(54.7023545, -3.2765753)",1038.0
9,GR,Greece,"(38.9953683, 21.9877132)",2.0


In [1]:
import folium
from folium.plugins import MarkerCluster
import math

europe = pd.read_csv("cleaned/TotalCasesEurope.csv")
america = pd.read_csv("cleaned/TotalCasesAmerica.csv")

world_map= folium.Map(tiles="cartodbpositron")
marker_cluster = MarkerCluster().add_to(world_map)

def addMarkers(df):
    for i in range(len(df)):
        if not isinstance(df.iloc[i]['Location'], str):
            continue
        lat = df.iloc[i]['Location'].split(", ")[0][1:]
        long = df.iloc[i]['Location'].split(", ")[1][:-1]
        val = int(df.iloc[i]['NumValue']) if not math.isnan(df.iloc[i]['NumValue']) else 0
        #print(df.iloc[i]['RegionName'], lat, long, val)
        radius=0
        popup_text = """Country : {}<br> %of Users : {}<br>"""
        popup_text = popup_text.format(df.iloc[i]['RegionName'], val)
        
        folium.CircleMarker(location = [lat, long], radius=radius).add_to(marker_cluster)


addMarkers(europe)
addMarkers(america)

#addMarkers(europe[['CountryCode', "RegionName", "Location", "NumValue"]].groupby(['CountryCode', "RegionName", "Location"]).sum().reset_index())
#addMarkers(america[['CountryCode', "RegionName", "Location", "NumValue"]].groupby(['CountryCode', "RegionName", "Location"]).sum().reset_index())
world_map

NameError: name 'pd' is not defined

In [10]:
import plotly.express as px

europe = pd.read_csv("cleaned/TotalCasesEurope.csv")
europe = europe[['CountryCode', "RegionName", "Location", "NumValue"]].groupby(['CountryCode', "RegionName", "Location"]).sum().reset_index()

america = pd.read_csv("cleaned/TotalCasesAmerica.csv")
america = america[['CountryCode', "RegionName", "Location", "NumValue"]].groupby(['CountryCode', "RegionName", "Location"]).sum().reset_index()

df = pd.concat([europe, america])

fig = px.choropleth(df, locations="CountryCode",
                    color="NumValue", # lifeExp is a column of gapminder
                    hover_name="RegionName", # column to add to hover information
                    color_continuous_scale=px.colors.sequential.Plasma)
fig.show()

In [18]:
year = 2022

europe = pd.read_csv("cleaned/TotalCasesEurope.csv")
europe = europe[europe["Time"] == year]
europe = europe[['CountryCode', "RegionName", "Location", "NumValue"]].groupby(['CountryCode', "RegionName", "Location"]).sum().reset_index()

america = pd.read_csv("cleaned/TotalCasesAmerica.csv")
america = america[america["Time"] == year]
america = america[['CountryCode', "RegionName", "Location", "NumValue"]].groupby(['CountryCode', "RegionName", "Location"]).sum().reset_index()

df = pd.concat([europe, america])

In [19]:
df.sort_values(by=['NumValue'], inplace=True, ascending=False)
df

,CountryCode,RegionName,Location,NumValue
7,BRA,Brazil,"(44.6976778, 7.8542707)",265289.0
35,PRY,Paraguay,"(50.273814, 4.4326804)",2443.0
20,GTM,Guatemala,"(51.34473215, 12.285764313430338)",1800.0
33,PER,Peru,"(39.6370289, -82.2044367)",595.0
36,SLV,El Salvador,"(37.26398255, -8.348731138182709)",154.0
11,COL,Colombia,"(50.938361, 6.959974)",94.0
23,HND,Honduras,"(22.3511148, 78.6677428)",44.0
12,CRI,Costa Rica,"(19.7589517, 99.6734592)",25.0
31,NIC,Nicaragua,"(40.9357176, 47.6645465)",4.0
32,PAN,Panama,"(35.4338536, 69.7930899)",2.0


In [22]:
import k3d
import vtk
import ipywidgets as widgets

reader = vtk.vtkGLTFReader() 
reader.SetFileName('sample_glb/5IRE.glb')
reader.Update() 

plot = k3d.plot()
mb = reader.GetOutput()

iterator = mb.NewIterator()

vtk_polyobjects = []
while not iterator.IsDoneWithTraversal():
    item = iterator.GetCurrentDataObject()
    vtk_polyobjects.append(item)
    iterator.GoToNextItem()

    
for obj in vtk_polyobjects:
    plot += k3d.vtk_poly_data(obj, color=0x222222)
plot.display()

Output()

In [30]:
import trimesh
import pyrender
fuze_trimesh = trimesh.load('sample_glb/5IRE.glb')
#mesh = pyrender.Mesh.from_trimesh(fuze_trimesh)
scene = pyrender.Scene.from_trimesh_scene(fuze_trimesh)
pyrender.Viewer(scene, use_raymond_lighting=True)

ValueError: Incorrect texture coordinate shape